In [ ]:
import gensim
import chardet
from gensim.models import Word2Vec

In [ ]:
try:
    with open('reviews_data.txt', 'r', encoding='utf-8', errors='replace') as file:
        documents = file.readlines()
except FileNotFoundError:
    print("Error")
    raise

doc_tok = [doc.lower().split() for doc in documents]

model = Word2Vec(
    sentences=doc_tok,   # Frases tokenizadas
    vector_size=150,                 # Dimensión de los vectores generados
    window=10,                       # Tamaño de la ventana de contexto
    min_count=2,                     # Frecuencia mínima de las palabras para ser incluidas
    workers=4                        # Número de threads a utilizar
)


In [ ]:
# Visualizar las palabras en el vocabulario
vocabulary = list(model.wv.index_to_key)
print("Primeras palabras en el vocabulario:", vocabulary[:10])

# Obtener el vector de una palabra específica 
word = 'hotel'
if word in vocabulary:
    example_word_vector = model.wv[word]
    print(f"Vector para la palabra '{word}':", example_word_vector)
else:
    print(f"Palabra '{word}' no encontrada en el vocabulario")

# Encontrar las 10 palabras más similares

In [ ]:
palabras_iniciales = ["street", "good", "dog", "mother", "bed"]
palabras_similares = {}

for palabra in palabras_iniciales:
    try:
        similares = model.wv.most_similar(palabra, topn=10)
        palabras_similares[palabra] = [p[0] for p in similares]
        print(f"Palabras similares a '{palabra}': {palabras_similares[palabra], similares}")
    except KeyError:
        print(f"La palabra '{palabra}' no está en el vocabulario del modelo.")



In [ ]:
palabras_iniciales = ["street", "good", "dog", "mother", "bed"]
similar_words_dict = {}
palabras = set(palabras_iniciales)

for palabra in palabras_iniciales:
    if palabra in vocabulary:
        similar_words = model.wv.most_similar(palabra, topn=10)
        similar_words_dict[palabra] = similar_words
        palabras.update([w for w, _ in similar_words])
        
embeddings = {}
for palabra in palabras:
    if palabra in vocabulary:
        embeddings[palabra] = model.wv[palabra]

for palabra, vector in embeddings.items():
    print(f"Embedding para la palabra '{palabra}':", vector)

In [ ]:
# Aplicar PCA para reducir la dimensionalidad de los embeddings a 2 componentes principales
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

embedding_vectors = list(embeddings.values())
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embedding_vectors)

plt.figure(figsize=(12, 8))
for i, word in enumerate(embeddings.keys()):
    plt.scatter(reduced_embeddings[i, 0], reduced_embeddings[i, 1])
    plt.text(reduced_embeddings[i, 0] + 0.01, reduced_embeddings[i, 1] + 0.01, word, fontsize=9)

plt.title("Representación PCA de los Embeddings de Palabras")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.grid()
plt.show()